### Taking the infinite limit

Just following through the tutorial directly:

We take $\mathbf{Z}$ to be in left-ordered form, and write it as $[ \mathbf{Z}_+ \mathbf{Z}_0]$.

Therefore, $|\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}|$ becomes:

$$
\left|
  \begin{bmatrix}
    \mathbf{Z}^T_+ \mathbf{Z}_+ & \mathbf{0} \\
    \mathbf{0} & \mathbf{0} \\
  \end{bmatrix}
  +
  \frac{\sigma_X^2}{\sigma_A^2}
  \mathbf{I}_K
\right|
$$

$$
= \left(
  \frac{\sigma_X^2}{\sigma_A^2}
\right)^{K_0}
\left|
  \mathbf{Z}_+^T\mathbf{Z}_+
  +
  \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}_{K^+}
\right|
$$

And: $\mathbf{Z}(\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I})^{-1}\mathbf{Z}^T$ becomes:

$$
\mathbf{Z}_+
\left(
\mathbf{Z}^T_+
\mathbf{Z}_+
+
\frac{\sigma_X^2}{\sigma_A^2}
\mathbf{I}_{K_+}
\right)^{-1}
\mathbf{Z}_+^T
$$


Reminder, equation 21 from the tutorial is the marginal probability of $\mathbf{X}$ given $\mathbf{Z}$, with $\mathbf{A}$ marginalized out:

$$
P(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A)
=
\frac
  {1}
  {(2\pi)^{ND/2} \sigma_X^{(N-K)D} \sigma_A^{KD} |\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I}|^{D/2} }
\exp \left(
  -\frac{1}{2\sigma_X^2}
  \mathrm{tr} \left(
    \mathbf{X}^T(\mathbf{I} - \mathbf{Z}(\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I})^{-1}\mathbf{Z}^T)\mathbf{X}
  \right)
\right)
$$

$$
=
\frac
  {1}
  {(2\pi)^{ND/2} \sigma_X^{(N-K_+)D} \sigma_A^{K_+D} |\mathbf{Z_+}^T\mathbf{Z_+} + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I}_{K_+}|^{D/2} }
\exp \left(
  -\frac{1}{2\sigma_X^2}
  \mathrm{tr} \left(
    \mathbf{X}^T(\mathbf{I} - \mathbf{Z}_+(\mathbf{Z}_+^T\mathbf{Z}_+ + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I}_{K+})^{-1}\mathbf{Z}_+^T)\mathbf{X}
  \right)
\right)
$$